<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/4_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# DOSYA ADI: egitim_stacking.py
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import joblib
import xgboost as xgb
import lightgbm as lgb
import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# --- EKLENEN METRİKLER ---
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, cohen_kappa_score, classification_report

# --- 1. ADIM: VERİ ÜRETİMİ ---
print(">> Veri üretiliyor (50.000 Adet)...")

def veri_uret(n):
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(n):
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        risk = np.random.beta(2, 2)
        maf = np.random.exponential(0.05)
        cons = np.random.uniform(0, 10)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        score = (risk * 0.4) + (cons/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydro)/5 * 0.2)
        score += np.random.normal(0, 0.05)
        label = 1 if score > 0.65 else 0

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    df_feat = pd.DataFrame(bio_feats, columns=['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight'])
    return dna_seqs, prot_seqs, df_feat, np.array(labels)

dna_data, prot_data, X_num, y = veri_uret(50000)

# --- 2. ADIM: ÖN İŞLEME ---
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_data)
X_dna = pad_sequences(dna_tok.texts_to_sequences(dna_data), maxlen=11, padding='post')

prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_data)
X_prot = pad_sequences(prot_tok.texts_to_sequences(prot_data), maxlen=11, padding='post')

# --- 3. ADIM: VERİ BÖLME ---
X_dna_temp, X_dna_test, X_prot_temp, X_prot_test, X_num_temp, X_num_test, y_temp, y_test = train_test_split(
    X_dna, X_prot, X_num, y, test_size=0.2, random_state=42
)

X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna_temp, X_prot_temp, X_num_temp, y_temp, test_size=0.25, random_state=42
)

# --- 4. ADIM: MODELLERİN EĞİTİMİ ---
# CNN
in_dna = Input(shape=(11,))
emb_dna = Embedding(len(dna_tok.word_index)+1, 8)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_dna))

in_prot = Input(shape=(11,))
emb_prot = Embedding(len(prot_tok.word_index)+1, 8)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_prot))

in_num_layer = Input(shape=(6,))
x3 = BatchNormalization()(Dense(16, activation='relu')(in_num_layer))

merged = Concatenate()([x1, x2, x3])
z = Dropout(0.3)(Dense(32, activation='relu')(merged))
out = Dense(1, activation='sigmoid')(z)

model_cnn = Model(inputs=[in_dna, in_prot, in_num_layer], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(">> CNN Eğitiliyor...")
model_cnn.fit([X_dna_base, X_prot_base, X_num_base], y_base, epochs=5, batch_size=32, verbose=0)

# XGBoost
print(">> XGBoost Eğitiliyor...")
model_xgb = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, use_label_encoder=False, eval_metric='logloss')
model_xgb.fit(X_num_base, y_base)

# LightGBM
print(">> LightGBM Eğitiliyor...")
model_lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, verbose=-1)
model_lgbm.fit(X_num_base, y_base)

# Meta-Learner
print(">> Meta-Learner Eğitiliyor...")
pred_cnn_meta = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
pred_xgb_meta = model_xgb.predict_proba(X_num_meta)[:, 1]
pred_lgbm_meta = model_lgbm.predict_proba(X_num_meta)[:, 1]
X_stack_meta = np.column_stack((pred_cnn_meta, pred_xgb_meta, pred_lgbm_meta))

meta_model = LogisticRegression()
meta_model.fit(X_stack_meta, y_meta)

# --- 5. ADIM: DETAYLI TEST RAPORU ---
print("\n" + "="*50)
print("📊 DETAYLI PERFORMANS ANALİZİ")
print("="*50)

# Test seti tahminleri
p1 = model_cnn.predict([X_dna_test, X_prot_test, X_num_test], verbose=0).flatten()
p2 = model_xgb.predict_proba(X_num_test)[:, 1]
p3 = model_lgbm.predict_proba(X_num_test)[:, 1]
X_stack_test = np.column_stack((p1, p2, p3))

# Final Karar
final_preds = meta_model.predict(X_stack_test)

# Metrik Hesaplamaları
acc = accuracy_score(y_test, final_preds)
f1 = f1_score(y_test, final_preds)
prec = precision_score(y_test, final_preds)
rec = recall_score(y_test, final_preds)
kappa = cohen_kappa_score(y_test, final_preds)

print(f"✅ Accuracy (Doğruluk) : %{acc*100:.2f}")
print(f"🎯 Precision (Kesinlik): %{prec*100:.2f} (Hasta dediklerimin ne kadarı hasta?)")
print(f"🔍 Recall (Duyarlılık) : %{rec*100:.2f} (Hastaların ne kadarını yakaladık?)")
print(f"⚖️ F1 Score            : %{f1*100:.2f} (Denge Puanı)")
print(f"🎲 Cohen's Kappa       : {kappa:.4f}   (Şanstan arındırılmış tutarlılık)")

print("-" * 50)
print("Meta Model Ağırlıkları (Hangi modele güveniyor?):")
print(f"CNN: {meta_model.coef_[0][0]:.2f}, XGB: {meta_model.coef_[0][1]:.2f}, LGBM: {meta_model.coef_[0][2]:.2f}")
print("="*50)

# --- 6. ADIM: KAYIT ---
if not os.path.exists('model_stack'): os.makedirs('model_stack')
model_cnn.save('model_stack/base_cnn.h5')
joblib.dump(model_xgb, 'model_stack/base_xgb.pkl')
joblib.dump(model_lgbm, 'model_stack/base_lgbm.pkl')
joblib.dump(meta_model, 'model_stack/meta_model.pkl')
with open('model_stack/dna_tok.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok.pickle', 'wb') as f: pickle.dump(prot_tok, f)

# Test verisini kaydet
test_df = X_num_test.copy()
test_df['DNA_Dizisi'] = dna_tok.sequences_to_texts(X_dna_test)
test_df['DNA_Dizisi'] = test_df['DNA_Dizisi'].apply(lambda x: x.replace(' ', '').upper())
test_df['Protein_Dizisi'] = prot_tok.sequences_to_texts(X_prot_test)
test_df['Protein_Dizisi'] = test_df['Protein_Dizisi'].apply(lambda x: x.replace(' ', '').upper())
test_df['Gercek_Deger'] = y_test
test_df.to_csv('test_verileri_stacking.csv', index=False)

print("✅ Eğitim tamamlandı ve dosyalar kaydedildi.")

>> Veri üretiliyor (50.000 Adet)...
>> CNN Eğitiliyor...
>> XGBoost Eğitiliyor...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [01:03:14] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


>> LightGBM Eğitiliyor...
>> Meta-Learner Eğitiliyor...

📊 DETAYLI PERFORMANS ANALİZİ


✅ Accuracy (Doğruluk) : %88.87
🎯 Precision (Kesinlik): %83.75 (Hasta dediklerimin ne kadarı hasta?)
🔍 Recall (Duyarlılık) : %79.63 (Hastaların ne kadarını yakaladık?)
⚖️ F1 Score            : %81.64 (Denge Puanı)
🎲 Cohen's Kappa       : 0.7366   (Şanstan arındırılmış tutarlılık)
--------------------------------------------------
Meta Model Ağırlıkları (Hangi modele güveniyor?):
CNN: 2.07, XGB: 3.54, LGBM: 1.46
✅ Eğitim tamamlandı ve dosyalar kaydedildi.


In [59]:
# DOSYA ADI: test_stacking.py
import pandas as pd
import numpy as np
import pickle
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, cohen_kappa_score

print(">> Modeller yükleniyor...")

# 1. YÜKLEME
with open('model_stack/dna_tok.pickle', 'rb') as f: dna_tok = pickle.load(f)
with open('model_stack/prot_tok.pickle', 'rb') as f: prot_tok = pickle.load(f)
model_cnn = load_model('model_stack/base_cnn.h5')
model_xgb = joblib.load('model_stack/base_xgb.pkl')
model_lgbm = joblib.load('model_stack/base_lgbm.pkl')
meta_model = joblib.load('model_stack/meta_model.pkl')

# 2. TÜM TEST VERİSİNİ YÜKLE VE GENEL PERFORMANSI ÖLÇ
try:
    df_all = pd.read_csv('test_verileri_stacking.csv')
    print(f">> {len(df_all)} adet test verisi bulundu. Genel performans hesaplanıyor...")

    # Tüm veriyi hazırla
    all_dna = pad_sequences(dna_tok.texts_to_sequences(df_all['DNA_Dizisi']), maxlen=11, padding='post')
    all_prot = pad_sequences(prot_tok.texts_to_sequences(df_all['Protein_Dizisi']), maxlen=11, padding='post')

    # Sayısal veriler (Burada astype('float32') yaparak hatayı önlüyoruz)
    feat_cols = ['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight']
    all_feats = df_all[feat_cols].values.astype('float32')
    all_feats_df = pd.DataFrame(all_feats, columns=feat_cols)

    # Tahminler
    p1 = model_cnn.predict([all_dna, all_prot, all_feats], verbose=0).flatten()
    p2 = model_xgb.predict_proba(all_feats_df)[:, 1]
    p3 = model_lgbm.predict_proba(all_feats_df)[:, 1]

    # Stacking
    stack_all = np.column_stack((p1, p2, p3))
    preds_all = meta_model.predict(stack_all)
    y_true = df_all['Gercek_Deger'].values

    # Metrikleri Yazdır
    print("\n" + "="*45)
    print("📊 GÜNCEL TEST DOSYASI GENEL RAPORU")
    print("="*45)
    print(f"Accuracy : %{accuracy_score(y_true, preds_all)*100:.2f}")
    print(f"Precision: %{precision_score(y_true, preds_all)*100:.2f}")
    print(f"Recall   : %{recall_score(y_true, preds_all)*100:.2f}")
    print(f"F1 Score : %{f1_score(y_true, preds_all)*100:.2f}")
    k_score = cohen_kappa_score(y_true, preds_all)
    print(f"Kappa    : {k_score:.4f} ", end="")

    if k_score > 0.8: print("(Mükemmel Uyum 🌟)")
    elif k_score > 0.6: print("(İyi Uyum 👍)")
    elif k_score > 0.4: print("(Orta Uyum 😐)")
    else: print("(Kötü Uyum ⚠️)")
    print("="*45 + "\n")

except FileNotFoundError:
    print("UYARI: Test dosyası bulunamadı, genel rapor geçiliyor.")

# 3. TEKİL ÖRNEK ANALİZİ (RASTGELE BİR HASTA)
print(">> Rastgele bir hasta seçilip detaylı analiz ediliyor...")
hasta = df_all.sample(1).iloc[0]

raw_dna = hasta['DNA_Dizisi']
raw_prot = hasta['Protein_Dizisi']
# Tekil veriyi de float32 yapıyoruz
raw_feats = np.array([hasta[feat_cols].values]).astype('float32')

# Modele Hazırla
seq_dna = pad_sequences(dna_tok.texts_to_sequences([raw_dna]), maxlen=11, padding='post')
seq_prot = pad_sequences(prot_tok.texts_to_sequences([raw_prot]), maxlen=11, padding='post')
feat_df = pd.DataFrame(raw_feats, columns=feat_cols)

# Tahmin
prob_cnn = model_cnn.predict([seq_dna, seq_prot, raw_feats], verbose=0)[0][0]
prob_xgb = model_xgb.predict_proba(feat_df)[0][1]
prob_lgbm = model_lgbm.predict_proba(feat_df)[0][1]

stack_vec = np.array([[prob_cnn, prob_xgb, prob_lgbm]])
final_prob = meta_model.predict_proba(stack_vec)[0][1]
final_karar = 1 if final_prob > 0.5 else 0

print(f"🧪 TEKİL HASTA ANALİZİ")
print("-" * 30)
print(f"DNA: {raw_dna} | Protein: {raw_prot}")
print(f"Biyo-İşaretler: {raw_feats[0]}")
print(f"🔍 CNN: %{prob_cnn*100:.1f} | XGB: %{prob_xgb*100:.1f} | LGBM: %{prob_lgbm*100:.1f}")
print(f"🎯 SİSTEM KARARI: {'HASTA ⚠️' if final_karar==1 else 'SAĞLIKLI ✅'} (Güven: %{final_prob*100:.2f})")
print(f"✅ GERÇEK DURUM : {'HASTA ⚠️' if hasta['Gercek_Deger']==1 else 'SAĞLIKLI ✅'}")

>> Modeller yükleniyor...
>> 10000 adet test verisi bulundu. Genel performans hesaplanıyor...

📊 GÜNCEL TEST DOSYASI GENEL RAPORU
Accuracy : %88.87
Precision: %83.75
Recall   : %79.63
F1 Score : %81.64
Kappa    : 0.7366 (İyi Uyum 👍)

>> Rastgele bir hasta seçilip detaylı analiz ediliyor...
🧪 TEKİL HASTA ANALİZİ
------------------------------
DNA: TAAGACGTGCG | Protein: PREGEPGWYNK
Biyo-İşaretler: [  0.60945106   0.09236953   7.1237082   -1.0393792    1.9973509
 -33.173462  ]
🔍 CNN: %25.9 | XGB: %11.1 | LGBM: %9.5
🎯 SİSTEM KARARI: SAĞLIKLI ✅ (Güven: %7.45)
✅ GERÇEK DURUM : SAĞLIKLI ✅
